# Part 2: Einführung in das Federated Learning

In dem letzten Kapitel wurden *PointerTensors* eingeführt, welche das nötige Grundgerüst für *Privacy-Preserving (privatsphäre-erhaltend) Deep Learning* darstellen. In diesem Kapitel lernen wir wie diese grundlegenden Werkzeuge uns dabei helfen unseren ersten privatsphäre-erhaltenden Deep Learning Algorithmus zu implementieren, *Federated Learning*. 

Authoren:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

### Was ist Federated Learning?

Federated Learning ist eine einfache, mächtige Art um Deep Learning Modelle zu trainieren. Wenn du über Trainingsdaten nachdenkst, sind diese immer das Resultat eines speziellen Datensammlungsprozesses. Personen generieren Daten indem sie über ihre Geräte reale Ereignisse in der Welt aufnehmen. Normalerweise werden alle diese Daten an einen einzelnen, zentralen Ort gebracht um dort ein Machine Learning Model zu trainieren. 
Federated Learning stellt dieses Konzept komplett auf dem Kopf!

Anstelle die Trainingsdaten zu dem zu trainierenden Model zu bringen (auf einem zentralen Server), wird das Model nun zu den Trainingsdaten gebracht (Wo auch immer sich diese befinden)

Die Grundidee dabei ist, dass der Datenbesitzer die einzige Person bleibt, die eine permanente Kopie der Daten hat (seiner Daten hat) und so die Kontrolle darüber behält wer Zugriff auf diese Daten hat bzw. wer diese zu sehen bekommt. 
Nicht schlecht oder? 

# Kapitel 2.1 - Ein einfaches Federated Learning Beispiel 

Fangen wir an ein einfaches Beispiel Model zentralisiert zu trainieren. Der Einfachkeitshalber verwenden wir ein so einfach wie mögliches Model. Dazu brauchen wir: 

- Ein Beispiel Datensatz 
- Ein Model 
- Grundlegende Trainingslogik um das Model auf den Daten zu trainieren

Hinweis: Falls dir diese API unbekannt vorkommt schau einfach mal bei [fast.ai](http://fast.ai) vorbei und schau dir deren Kurse an bevor du dieses Tutorial weiter verfolgst. 

In [ ]:
import torch
from torch import nn
from torch import optim

In [ ]:

# Ein Beispiel Datensatz
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# Ein einfaches Beispiel-Model 
model = nn.Linear(2,1)

def train():
    # Trainingslogik
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) Lösche vorherige Gradienten (falls vorhanden)
        opt.zero_grad()

        # 2) Mache eine Prognose mittels des Models
        pred = model(data)

        # 3) Berechne den Fehler (Wie weit wir daneben lagen)
        loss = ((pred - target)**2).sum()

        # 4) Finde heraus welche Gewichte in unserem Model dazu beigetragen haben, dass es ein Fehler gab
        loss.backward()

        # 5) Ändere diese Gewichte 
        opt.step()

        # 6) Gebe den Fortschritt aus 
        print(loss.data)

In [ ]:
train()

Und fertig sind wir! Wir haben ein einfaches Model mittels des Standard-Trainingsprozesses trainiert. Der komplette Datensatz ist auf unserem lokalen Rechner gespeichert und kann benutzt werden um unser Model zu verbessern. 
Federated Learning funktioniert jedoch anders. 
Also, lass uns dieses Beispiel nun mittels Federated Learning durchführen. 

Also, was brauchen wir: 

- Erstelle ein paar *worker* (Arbeiter)
- Auf jedem worker speichere einen *Pointer* (Zeiger) auf den jeweiligen Trainingsdatensatz 
- Verändere die Trainingslogik um Federated Learning zu machen

    Neue Trainingsschritte: 
    - Sende Model zu dem richtigen worker 
    - Trainiere auf den dort vorhanden Daten 
    - Gebe des Model zurück und fahre mit nächsten worker fort

In [ ]:
import syft as sy
hook = sy.TorchHook(torch)


In [ ]:
# Erstelle ein paar worker

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")


In [ ]:
# Ein Beispiel Datensatz 
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# Speichere einen pointer auf den jeweiligen Datensatz auf jedem Arbeiter 
# indem du Trainingsdaten zu bob und alice sendest 
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Initialisiere das Beispiel Model 
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# Organisiere die Pointer in einer Liste 
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

In [ ]:
from syft.federated.floptimizer import Optims
workers = ['bob', 'alice']
optims = Optims(workers, optim=optim.Adam(params=model.parameters(),lr=0.1))

In [ ]:
def train():
    # Trainingslogik
    for iter in range(10):
        
        # NEU) iteriere durch den Datensatz von jedem worker 
        for data,target in datasets:
            
            # NEU) Sende Model zu korrektem worker
            model.send(data.location)
            
            # Rufe durch get_optim den optimizer für den worker auf 
            opt = optims.get_optim(data.location.id)
            #print(data.location.id)

            # 1) Lösche vorherige Gradienten (falls vorhanden) 
            opt.zero_grad()

            # 2) Mache eine Prognose 
            pred = model(data)

            # 3) Berechen den Fehler 
            loss = ((pred - target)**2).sum()

            # 4) Finde heraus welche Gewichte im Model zu dem Fehler beigtragen haben 
            loss.backward()

            # 5) Ändere diese Gewichte
            opt.step()
            
            # NEU) Erhalte Model zurück (mit Gradienten)
            model.get()

            # 6) Gebe den Fortschritt aus 
            print(loss.get()) # NEU) kleine Änderung... müssen .get() von loss aufrufen 
    
# federated averaging

In [ ]:
train()

## Klasse!

Und voilà! Wir sind nun dabei ein sehr einfaches Deep Learning Model mittels Federated Learning zu trainieren! Wir senden das Model zu jedem worker, berechnen die Gradienten, und schicken die Gradienten dann zu unserem lokalen Rechner zurück, wo wir die Modelparameter entsprechend ändern. *Niemals* während des gesamten Prozesses sehen wir oder fordern wir Zugriff auf die verschiedenen Datensätze auf deren Basis wir die Modelparameter ändern. 
Wir erhalten so die Privatsphäre von Bob und Alice!! 

## Grenzen dieses Beispiels 

Obowhl dieses Beispiel eine gute Einführung das Federated Learning bietet, hat es jedoch ein paar wichtige Grenzen. Vorallem, wenn wir `model.get()` aufrufen und das veränderte Model von Bob oder Alice bekommen, können wir tatsächlich viel über deren Trainingsdatensatz in Erfahrung bringen indem wir uns einfach die berechneten Gradienten anschauen. 
Teilweise kann so sogar der komplette Trainingsdatensatz wiederhergestellt werden!

Was können wir also dagegen machen? Die erste Strategie gegen dieses Problem ist **über die verschiedenen Gradienten mehrerer Individuen den Durschnitt zu bilden und diesen auf den zentralen Server hochzuladen.** Diese Strategie erfordert jedoch eine etwas fortgeschrittene Benutzung von PointerTensor Objekten. 
Deswegen werden wir uns im nächsten Kapitel etwas fortgeschrittene Funktionalitäten von Pointern anschauen und damit dieses Federated Learning Beispiel verbessern.  


# Glückwunsch!!! - Zeit um der Gemeinschaft beizutreten! 

Glückwunsch für die Vollendung dieses Tutorials! Falls dir das Tutorial gefallen hat und du der Gemeinschaft für privatsphäre-erhaltendem, dezentralisiertem Besitz von KI und der KI "Lieferkette" (Daten) beitreten möchtest, kannst du das gerne durch folgende Wege machen! 

### Star PySyft auf GitHub

Der einfachste Weg um unserer Gemeinschaft zu helfen ist eines der Repos auf Github zu "star[n]"! Das hilft uns Aufmerksamkeit für die coolen Tools, die wir bauen, zu sammeln.

- [Star PySyft](https://github.com/OpenMined/PySyft)


### Tritt unserem Slack bei! 

Der beste Weg um über die neuesten Errungenschaften unserer Gemeinschaft zu erfahren ist unserer Gemeinschaft beizutreten! Das kannst du indem du folgendes Formular ausfüllst:  [http://slack.openmined.org](http://slack.openmined.org)


### Werde Mitglied eines Programmierprojektes!

Der beste Weg um dich an unserer Gemeinschaft zu beteiligen ist Programmier-Mitwirkender zu werden (Contributer)! Zu jeder Zeit kannst du zu der Issue Seite von PySyft auf GitHub gehen und nach "Projects" filtern. Das wird dir alle top level tickets anzeigen und somit kannst du einen guten Überblick bekommen über die Projekte, denen du beitreten kannst! Falls du keinem Projekt beitreten möchtest, aber trotzdem ein bisschen Programmieren möchtest, kannst du auch nach kleineren Mini-Projekten Ausschau halten, die mit "good first issue" gekennzeichnet sind.  

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Spenden

Wenn du keine Zeit hast dich an unserer Programmierarbeit zu beteiligen, du uns aber trotzdem unterstützen möchtest, kannst du ein Spender unserer Open Collective werden. Alle Spenden werden für Web Hosting und anderen Gemeinschaftsausgaben, wie z.B Hackathons und Meetups benutzt! 

[OpenMined's Open Collective Page](https://opencollective.com/openmined)